<a href="https://colab.research.google.com/github/park-geun-hyeong/CUDA/blob/main/EX3_0923.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Error Handling

In [16]:
!nvidia-smi

Fri Sep 23 04:52:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


### enum cudaError_t()
- cuda 함수 호출 시 성공 여부를 저장하는 리턴값
- cuda 함수 호출 성공시 : cudaSuccess(==0) return
- cuda 함수 호출 실패시: 수십가지의 Error value중 적절한 값을 리턴

### const char* cudaGetErrorName(cudaError_t error)
- error의 이름을 return
- error 코드가 인식되지 않는다면 'unrecognized error code' return

### const char* cudaGetErrorString(cudaError_t error)
- error에 대한 설명을 return
- error 코드가 인식되지 않는다면 'unrecognized error code' return



In [5]:
%%writefile cudaErrExample.cu

#include<stdio.h>

int main(){

  const char* err1 = cudaGetErrorName(cudaErrorMemoryAllocation);
  const char* err1_explain = cudaGetErrorString(cudaErrorMemoryAllocation);
  printf("%s: ", err1);
  printf("%s\n", err1_explain);

  const char* err2 = cudaGetErrorName(cudaErrorInvalidValue);
  const char* err2_explain = cudaGetErrorString(cudaErrorInvalidValue);
  printf("%s: ", err2);
  printf("%s\n", err2_explain);

  return 0;
}


Overwriting cudaErrExample.cu


In [6]:
!ls

cudaErrExample	cudaErrExample.cu  sample_data


In [7]:
!nvcc -o cudaErrExample cudaErrExample.cu

In [8]:
!./cudaErrExample

cudaErrorMemoryAllocation: out of memory
cudaErrorInvalidValue: invalid argument


### cudaError_t cudaGetLastError(void)
- 동일 호스트 스레드 내의 runtime call 로부터 발생된마지막 에러를 리터후 마지막 에러값을 cudaSuccess로 리셋함

### cudaError_Tt cudaPeekAtLastError(void)
- 동일 호스트 스레드 내의 runtime call 로부터 발생된마지막 에러를 리터후 마지막 에러값을 cudaSuccess로 리셋하지 않음

### 내장 메크로
- __FILE__ : 현재 소스파일의 경로
- __LINE__ : 코드 라인번호(정수형)


In [9]:
%%writefile cudaErrMacro.cu

#include<stdio.h>

int main(){

  cudaError_t err = cudaErrorMemoryAllocation;
  // cudaError_t err = cudaGetLastError();

  if(err != cudaSuccess){
    fprintf(stderr, "[cuda error] %s (line %d): %s(%s)\n", __FILE__, __LINE__, cudaGetErrorName(err),cudaGetErrorString(err));
    exit(err);
  }

  return 0;
}

Writing cudaErrMacro.cu


In [10]:
!nvcc -o cudaErrMacro cudaErrMacro.cu

In [11]:
!./cudaErrMacro

[cuda error] cudaErrMacro.cu (line 10): cudaErrorMemoryAllocation(out of memory)


In [32]:
%%writefile cudaErrMacro2.cu

#include<stdio.h>

#undef NO_DEBUG

#ifdef NO_DEBUG   // release mode
  #define CUDA_ERR_CHECK(ans) (ans)
#else             // debug mode
  #define CUDA_ERR_CHECK(ans) {cudaErrCheck((ans), __FILE__, __LINE__);}
  inline void cudaErrCheck(cudaError_t err, const char *file, int line, bool abort=true)
  {
    if(err!=cudaSuccess)
    {
      fprintf(stderr, "[cuda error] %s (line %d): %s(%s)\n", file, line, cudaGetErrorName(err),cudaGetErrorString(err));
      if(abort) exit(err);
    }
  }
#endif

int main(){

  CUDA_ERR_CHECK(cudaErrorMemoryAllocation);
  return 0;
}

Overwriting cudaErrMacro2.cu


In [33]:
!nvcc -o cudaErrMacro2 cudaErrMacro2.cu

In [34]:
!./cudaErrMacro2

[cuda error] cudaErrMacro2.cu (line 22): cudaErrorMemoryAllocation(out of memory)


#### EX2에서 진행했던 memory copy example에서 고의적으로 direction error 발생시켜서 log 찍어보기


In [26]:
%%writefile memory_copy_err_check.cu

#include<stdio.h>

#undef NO_DEBUG

#ifdef NO_DEBUG   // release mode
  #define CUDA_ERR_CHECK(ans) (ans)
#else             // debug mode
  #define CUDA_ERR_CHECK(ans) {cudaErrCheck((ans), __FILE__, __LINE__);}
  inline void cudaErrCheck(cudaError_t err, const char *file, int line, bool abort=true)
  {
    if(err!=cudaSuccess)
    {
      fprintf(stderr, "[cuda error] %s (line %d): %s(%s)\n", file, line, cudaGetErrorName(err),cudaGetErrorString(err));
      if(abort) exit(err);
    }
  }
#endif

int main(){

  int size = 10;
  int src_data[size] = {0,1,2,3,4,5,6,7,8,9};
  int dst_data[size] = {0,0,0,0,0,0,0,0,0,0};

  printf("src data = ");
  for(int i = 0 ; i<size; i++){
    printf("%d ", src_data[i]);
  }
  printf("\n");

  int* dev_src = 0;
  int* dev_dst = 0;

  CUDA_ERR_CHECK(cudaMalloc((void**)&dev_src, size*sizeof(int)));
  CUDA_ERR_CHECK(cudaMalloc((void**)&dev_dst, size*sizeof(int)));
 
  // Host--> Device(Device -> Device로 고의적인 Error 발생!)
  CUDA_ERR_CHECK(cudaMemcpy(dev_src, src_data, size*sizeof(int), cudaMemcpyDeviceToDevice));
 
  // Device--> Device
  CUDA_ERR_CHECK(cudaMemcpy(dev_dst, dev_src, size*sizeof(int), cudaMemcpyDeviceToDevice));
  
  // Device--> Host
  CUDA_ERR_CHECK(cudaMemcpy(dst_data, dev_dst, size*sizeof(int), cudaMemcpyDeviceToHost));

  CUDA_ERR_CHECK(cudaFree(dev_src));
  CUDA_ERR_CHECK(cudaFree(dev_dst));
 
  printf("dst data = ");
  for(int i = 0 ; i<size; i++){
    printf("%d ", dst_data[i]);
  }
  printf("\n");

  return 1;
}

Overwriting memory_copy_err_check.cu


In [27]:
!nvcc -o memory_copy_err_check memory_copy_err_check.cu

In [28]:
!./memory_copy_err_check

src data = 0 1 2 3 4 5 6 7 8 9 
[cuda error] memory_copy_err_check.cu (line 41): cudaErrorInvalidValue(invalid argument)
